# SW 2 Signal flow grapher
See the following [repo](https://github.com/hanspi42/signalflowgrapher) for the enviornment settings.
<!-- ![alternative text](Figures/SW2_01.png) -->
<img src="Figures/SW2_01.png" 
     align="center" 
     width="450" />

\begin{aligned}
& v(t)=\frac{1}{C} \int_0^t i(t) d t . \\
& V(s)=\frac{1}{s C} I(s) \quad \begin{array}{l}
\text{Laplace} \\
\text{Transform}
\end{array} \\
& I=S C \cdot V \\
& I=G \cdot V
\end{aligned}

## Procedure:

1. Analize the circuit

<img src="Figures/SW2_02.png" 
     align="center" 
     width="450" />

2. Draw the voltage sources

<img src="Figures/SW2_03.png" 
     align="center" 
     width="450" />

3. Draw the current sources everywhere where no voltage source is

<img src="Figures/SW2_04.png" 
     align="center" 
     width="450" />

4. Draw the graph

<img src="Figures/SW2_05.png" 
     align="center" 
     width="450" />

5. Start signal flow grapher with the following command `python .\src\main\python\main.py` in the 'signalflowgrapher' folder (make also sure the correct environment is activated `conda activate sfg`)
6. Draw the graph in the signal flow grapher:

<img src="Figures/SW2_06.png" 
     align="center" 
     width="450" />

7. Copy maison, see below

In [1]:
from sympy import *
import sympy as sp
import numpy as np
import hanspitools as ht
import matplotlib.pyplot as plt
Delta = sp.symbols('Delta')
L1 = sp.symbols('L1')
T_num = sp.symbols('T_num')
T_den = sp.symbols('T_den')
T_io = sp.symbols('T_io')
G_1,G_2,A = sp.symbols('G_1,G_2,A')

loops = [(L1, -A*G_2/(G_1 + G_2))]
determinant = [(Delta, 1 - L1)]
denominator = [(T_den, Delta)]

P1,D1 = sp.symbols('P1,D1')
paths = [(P1, -A*G_1/(G_1 + G_2)), (D1, 1)]
numerator = [(T_num, D1*P1)]

transfer_function = [(T_io, T_num/T_den)]
T=T_io.subs(transfer_function).subs(numerator).subs(denominator).subs(determinant).subs(paths).subs(loops).simplify()
display(T)

-A*G_1/(A*G_2 + G_1 + G_2)

In [2]:
s=sp.symbols('s')
def niceT(T):
    return ht.mani.numden(lambda p: ht.mani.nicepoly(p,s),T)
def keeper(f,v):
    return sp.limit(f/v,v,sp.oo)*v
f=10.0**np.arange(1,8,0.01)
wn = 2*sp.pi*f
comp_val= [(G_1, 1), (G_2, 1), (A, 10000)]
T_a = ht.freq.amplitude(T,s,wn,comp_val)
print(T_a)
# T_p = ht.freq.phase(T,s,wn,comp_val)

0.9998000399920016


____

## Exercises
### 1.24

<p align="center">
<img src="Figures/SW2_10.png"
     width=60% />
</p>

1. First of all one has to redraw the circuit to read it in an easier way:

<p align="center">
<img src="Figures/SW2_11.png"
     width=40% />
</p>

2. The one can draw the circuit in the signal flow grapher:

<p align="center">
<img src="Figures/SW2_12.png"
     width=40% />
</p>

Here we assumed that we have finite gain. 

In [3]:
# Plot the equation we got from the signal flow grapher
import sympy as sp
import numpy as np
import hanspitools as ht
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets.embed import embed_minimal_html
def niceT(T,s):
    return ht.mani.numden(lambda p: ht.mani.nicepoly(p,s),T)
def di(x):
    display(x)
    # print(latex(x))
def sigdig(expr, num_digits):
    return expr.xreplace({n.evalf() : n if type(n)==int else Float(n, num_digits) for n in expr.atoms(Number)})
def eseries(v, E):
    return 10**((log(v, 10)*E).round()/E)
Delta = sp.symbols('Delta')
L1 = sp.symbols('L1')
T_num = sp.symbols('T_num')
T_den = sp.symbols('T_den')
T_io = sp.symbols('T_io')
A,G_1,G_3,k,G_2 = sp.symbols('A,G_1,G_3,k,G_2')

loops = [(L1, -A*G_2/(G_1 + G_2))]
determinant = [(Delta, 1 - L1)]
denominator = [(T_den, Delta)]

P1,D1,P2,D2 = sp.symbols('P1,D1,P2,D2')
paths = [(P1, -A*G_1/(G_1 + G_2)), (D1, 1), (P2, A*G_3/((1 - k)*(G_3/(1 - k) + G_3/k))), (D2, 1)]
numerator = [(T_num, D1*P1 + D2*P2)]

transfer_function = [(T_io, T_num/T_den)]
T_1=T_io.subs(transfer_function).subs(numerator).subs(denominator).subs(determinant).subs(paths).subs(loops).simplify()
display(T_1)


-A*(G_1 - k*(G_1 + G_2))/(A*G_2 + G_1 + G_2)

In [4]:
#Lets plot the gain
from IPython.display import display
def plot_gain(potentionmeter=0.5,r2_over_r1=1, R_3=1e3, gain=10000):
    """A function to plot the gain of the amplifier"""
    plt.figure(figsize=(10,2))
    R_1=1e3
    R_2=R_1*r2_over_r1
    comp_val= [(G_1, 1/R_1), (G_2, 1/R_2), (G_3, 1/R_3), (A, gain), (k, potentionmeter)]
    Ta = T_1.subs(comp_val)
    plt.plot(Ta,0, 'bo')
    plt.title('gain plot')
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    plt.grid()
    plt.show()
slider=widgets.interact(plot_gain,potentionmeter=(0.0,1),r2_over_r1=(0.1,2),R_3=(1e2,1e4),gain=(1e1,1e5))
# embed_minimal_html('export.html', views=[slider], title='Widgets export')

interactive(children=(FloatSlider(value=0.5, description='potentionmeter', max=1.0), FloatSlider(value=1.0, de…

3. With invinite gain the graph would look like the following:

<p align="center">
<img src="Figures/SW2_13.png"
     width=40% />
</p>

In [5]:
# Plot the equation we got from the signal flow grapher
Delta = sp.symbols('Delta')
L1,L2 = sp.symbols('L1,L2')
T_num = sp.symbols('T_num')
T_den = sp.symbols('T_den')
T_io = sp.symbols('T_io')
G_3,k,G_1,G_2 = sp.symbols('G_3,k,G_1,G_2')

loops = [(L1, -G_2/(G_1 + G_2)), (L2, 1)]
determinant = [(Delta, -L1 - L2 + 1)]
denominator = [(T_den, Delta)]

P1,D1,P2,D2 = sp.symbols('P1,D1,P2,D2')
paths = [(P1, -G_1/(G_1 + G_2)), (D1, 1), (P2, G_3/((1 - k)*(G_3/(1 - k) + G_3/k))), (D2, 1)]
numerator = [(T_num, D1*P1 + D2*P2)]

transfer_function = [(T_io, T_num/T_den)]
T=T_io.subs(transfer_function).subs(numerator).subs(denominator).subs(determinant).subs(paths).subs(loops).simplify()
display(T)
niceT(T.expand(), k)

(-G_1 + k*(G_1 + G_2))/G_2

-G_1/G_2 + k*(G_1 + G_2)/G_2

From the euqation above we see that the description in the exercise is true when $R_1=R_2 \Rightarrow G_1=G_2$

- when $k=0 \Rightarrow -\frac{G_1}{G_2}=-1$ 
- when $k=1 \Rightarrow -\frac{G_1}{G_2}+ \frac{k(G_1+G_2)}{G_2}=1$ 

#### (b) Additional resistor only changes one branch in the SFG

The function with finite gain look therefore like the following:

In [6]:
import sympy as sp
Delta = sp.symbols('Delta')
L1 = sp.symbols('L1')
T_num = sp.symbols('T_num')
T_den = sp.symbols('T_den')
T_io = sp.symbols('T_io')
A,G_1,G_3,k,G_4,G_2 = sp.symbols('A,G_1,G_3,k,G_4,G_2')

loops = [(L1, -A*G_2/(G_1 + G_2 + G_4))]
determinant = [(Delta, 1 - L1)]
denominator = [(T_den, Delta)]

P1,D1,P2,D2 = sp.symbols('P1,D1,P2,D2')
paths = [(P1, -A*G_1/(G_1 + G_2 + G_4)), (D1, 1), (P2, A*G_3/((1 - k)*(G_3/(1 - k) + G_3/k))), (D2, 1)]
numerator = [(T_num, D1*P1 + D2*P2)]

transfer_function = [(T_io, T_num/T_den)]
A_f=T_io.subs(transfer_function).subs(numerator).subs(denominator).subs(determinant).subs(paths).subs(loops).simplify()
display(A_f)
def plot_gain(potentionmeter=0.5,r2_over_r1=1, R_3=1e3, R_4=1e3,gain=10000):
    #create matplotlib figure with size 10x10
    plt.figure(figsize=(10,2))
    R_1=1e3
    R_2=R_1*r2_over_r1
    comp_val= [(G_1, 1/R_1), (G_2, 1/R_2), (G_3, 1/R_3), (G_4, 1/R_4), (A, gain), (k, potentionmeter)]
    Ta = A_f.subs(comp_val)
    plt.plot(Ta,0, 'bo')
    plt.title('gain plot')
    plt.xlim(-6,6)
    plt.ylim(-1,1)
    plt.grid()
    plt.show()
widgets.interact(plot_gain,potentionmeter=(0.0,1),r2_over_r1=(0.1,2),R_3=(1e2,1e4),R_4=(1e2,1e4),gain=(1e1,1e5))

-A*(G_1 - k*(G_1 + G_2 + G_4))/(A*G_2 + G_1 + G_2 + G_4)

interactive(children=(FloatSlider(value=0.5, description='potentionmeter', max=1.0), FloatSlider(value=1.0, de…

<function __main__.plot_gain(potentionmeter=0.5, r2_over_r1=1, R_3=1000.0, R_4=1000.0, gain=10000)>

2. With invinite gain like the following

In [7]:
import sympy as sp
Delta = sp.symbols('Delta')
L1,L2 = sp.symbols('L1,L2')
T_num = sp.symbols('T_num')
T_den = sp.symbols('T_den')
T_io = sp.symbols('T_io')
G_3,k,G_4,G_1,G_2 = sp.symbols('G_3,k,G_4,G_1,G_2')

loops = [(L1, -G_2/(G_1 + G_2 + G_4)), (L2, 1)]
determinant = [(Delta, -L1 - L2 + 1)]
denominator = [(T_den, Delta)]

P1,D1,P2,D2 = sp.symbols('P1,D1,P2,D2')
paths = [(P1, -G_1/(G_1 + G_2 + G_4)), (D1, 1), (P2, G_3/((1 - k)*(G_3/(1 - k) + G_3/k))), (D2, 1)]
numerator = [(T_num, D1*P1 + D2*P2)]

transfer_function = [(T_io, T_num/T_den)]
T_4=T_io.subs(transfer_function).subs(numerator).subs(denominator).subs(determinant).subs(paths).subs(loops).simplify()
display(T_4)

(-G_1 + k*(G_1 + G_2 + G_4))/G_2

In [8]:
niceT(T_4.expand(), k)
A_min, A_max = [ T_4.subs(k, x) for x in range(2) ] 
di(A_min)
di(A_max)

-G_1/G_2

(G_2 + G_4)/G_2

3. Now we want to determine $G_1,G_2,G_4$ in such a way that the output gain is from -5V to 5V when varying $k$. For that one can use the euations above. When $k=0$ we have the first equation when $k=1$ the second one. Furthermore we can set one value since we have three independent ones $\Rightarrow$ lets say $R_1=10kOhm$. Due to that one can set up the following equation system:


In [9]:
R_1=10e3
eqs = [Eq(G_1, 1/R_1), Eq(A_min, -5), Eq(A_max, 5)]
di(eqs)
sols = solve(eqs, [G_1, G_2, G_4], dict=True)
di(sols)
for g, x in sols[0].items():
    print('1/%s = %g kOhm' % (g, 1/x/1000))

def plot_gain(potentionmeter=0.5, R_3=1e3):
    #create matplotlib figure with size 10x10
    plt.figure(figsize=(10,2))
    R_1=1/sols[0][G_1]
    R_2=1/sols[0][G_2]
    R_4=1/sols[0][G_4]
    comp_val= [(G_1, 1/R_1), (G_2, 1/R_2), (G_3, 1/R_3), (G_4, 1/R_4), (k, potentionmeter)]
    Ta = T_4.subs(comp_val)
    plt.plot(Ta,0, 'bo')
    plt.title('gain plot')
    plt.xlim(-5,5)
    plt.ylim(-1,1)
    plt.grid()
    plt.show()
widgets.interact(plot_gain,potentionmeter=(0.0,1),R_3=(1e2,1e4))


[Eq(G_1, 0.0001), Eq(-G_1/G_2, -5), Eq((G_2 + G_4)/G_2, 5)]

[{G_1: 0.000100000000000000,
  G_2: 2.00000000000000e-5,
  G_4: 8.00000000000000e-5}]

1/G_1 = 10 kOhm
1/G_2 = 50 kOhm
1/G_4 = 12.5 kOhm


interactive(children=(FloatSlider(value=0.5, description='potentionmeter', max=1.0), FloatSlider(value=1000.0,…

<function __main__.plot_gain(potentionmeter=0.5, R_3=1000.0)>